In [1]:
import numpy as np
import cv2 as cv

In [2]:
net=cv.dnn.readNet('yolo_req/yolov3.weights','yolo_req/yolov3_config.cfg')

In [3]:
classes = open('yolo_req/coco_lable.txt').read().strip().split("\n")

In [4]:
classes[:10]

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light']

In [5]:
cap=cv.VideoCapture(0)

while True:
    _,img=cap.read()
    #img=cv.imread('images/busy_street.jpg')

    height,width,depth=img.shape

    #image pre-proessing

    #normalizing by dividing with 255
    #converting BGR to RGB
    blob=cv.dnn.blobFromImage(img,1/255, (416,416), (0,0,0), swapRB=True, crop=False)

    net.setInput(blob)
    op_layers_names=net.getUnconnectedOutLayersNames()
    layer_outputs=net.forward(op_layers_names)

    boxes=[]
    predictions=[]
    class_ids=[]

    for op in layer_outputs:
        #the first 5 elements of the detected objs are the 4 co-ordinates of boxes and 1 accuracy of bounding box 
        #rest of the 80 elements are the pred score of different classes
        for detected in op:
            preds=detected[5:]
            class_id=np.argmax(preds)
            max_preds=preds[class_id]
            if max_preds>0.5:
                cx=int(detected[0]*width)
                cy=int(detected[1]*height)
                h=int(detected[2]*width)
                w=int(detected[3]*height)

                x=int(cx-w/2)
                y=int(cy-h/2)

                boxes.append([x,y,w,h])
                predictions.append(float(max_preds))
                class_ids.append(class_id)


    indexes=cv.dnn.NMSBoxes(boxes,predictions,0.5,0.4)

    colors=np.random.uniform(0,255,size=(len(boxes),3))
    
    if len(indexes)>0:
        for i in indexes.flatten():
            x,y,w,h=boxes[i]
            lable=str(classes[class_ids[i]])
            text="{}, {:.2f}%".format(lable,predictions[i]*100)
            color=colors[i]
            cv.rectangle(img,(x,y),(x+w,y+h),color,2)
            cv.putText(img,text, (x,y+20), cv.FONT_HERSHEY_PLAIN, 2, (255,255,255), 2)

        cv.imshow('Image',img)
        key=cv.waitKey(1)
        if key==27:
            break
        
cap.release()   
cv.destroyAllWindows()





